# Static LCA

This notebook explains how to conduct a static (i.e. deterministic; or in other words, ignoring uncertainties) LCA for the pavement structure defined in the case study used in the lectures.

## Content

3.1) Single-LCA: Presentation of the steps needed to calcule the environmental impacts for a product system corresponding to the initial pavement structure.

3.2) Multi-LCA

In [6]:
import bw2analyzer as bwa
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import pandas as pd
from bw2data.query import Filter, Query
from IPython.display import display

In [9]:
#Importing the variables with the project name and background db
from project_details import ei_name, project_name
print(ei_name)
print(project_name)

ecoinvent-391-cutoff
2024_STI_BW25_Lecture


In [10]:
bd.projects.set_current(project_name)
bd.databases

Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-391-cutoff

In [15]:
# Is the background database name the same as the one we wrote in `project_details.py`?
assert ei_name in bd.databases

In [16]:
fg_name = "asphalt"

In [17]:
# Assign the foreground database to a variable:
fgdb = bd.Database(fg_name)

## 3.1) Single-LCA

## Product system definition

We are going to create a product system for the initial pavement structure. In our foreground database the activity "pavement structure" has as reference product the inital pavement structure for which we want to calculate the environmental impacts related to the phases A1-A5. Let's selected it and assign it to the variable "pavement_structure"

In [19]:
pavement_structure = fgdb.get("pavement_structure_1")

In [20]:
bwa.print_recursive_supply_chain(pavement_structure, max_level=1)

1: 'pavement structure, AC Surf, 30% RAP, regular bitumen' (kilometer, NL, None)
  2.01e+06: 'AC Surf, 30% RAP, regular bitumen' (kilogram, NL, None)
  2.72e+06: 'AC Bin' (kilogram, NL, None)
  1.35e+07: 'AC Bin' (kilogram, NL, None)


## Functional unit 

The next step consists of defining the functional unit. This is done by using a dictionary with keys = activities and values = amounts. 
In our case study, the name of the activity is 'pavement_structure' and the amount is 1.

In [21]:
fu = {pavement_structure:1}
fu

{'pavement structure, AC Surf, 30% RAP, regular bitumen' (kilometer, NL, None): 1}

## LCIA Methods

n this step we are going to select the LCIA method to be used for calculating the environmental impact scores. In the notebook 2a_Background we explore the different LCIA methods that come with the databse 'biosphere3'. In this example we are going to use the method 'CML v4.8 2016'.

In [22]:
cml_methods = [m for m in bd.methods if m[0] == "CML v4.8 2016"]
cml_methods

[('CML v4.8 2016',
  'acidification',
  'acidification (incl. fate, average Europe total, A&B)'),
 ('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)'),
 ('CML v4.8 2016',
  'ecotoxicity: freshwater',
  'freshwater aquatic ecotoxicity (FAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: marine',
  'marine aquatic ecotoxicity (MAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: terrestrial',
  'terrestrial ecotoxicity (TETP inf)'),
 ('CML v4.8 2016',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('CML v4.8 2016', 'eutrophication', 'eutrophication (fate not incl.)'),
 ('CML v4.8 2016', 'human toxicity', 'human toxicity (HTP inf)'),
 ('CML v4.8 2016',
  'material resources: metals/minerals',
  'abiotic depletion potential (ADP): elements (ultimate reserves)'),
 ('CML v4.8 2016',
  'ozone depletion',
  'ozone layer depletion (ODP steady state)'),
 ('CML v4.8 2016',
  'photochemical oxidant formation',
  'photochemical oxidation (h

## Scores (i.e. results) calculation

In this step we are going to calculate the environmental impact scores. To do so, we need to do the following:

In [23]:
# Create a lca object
lca = bc.LCA(fu)

# Builds matrices, solves the system, generates an LCI matrix.
lca.lci()

results = [] #this list will store the LCA scores
for method in cml_methods:
    lca.switch_method(method)
    lca.lcia() # Characterization, i.e. the multiplication of the elements of the LCI matrix with characterization factors from the chosen LCIA method
    results.append((method[1].title(), 
                    lca.score, # Returns the score, i.e. the sum of the characterized inventory
                    bd.methods.get(method).get('unit')))
    print("Score is {:f} {} for category {}".format(lca.score, 
                                                 bd.methods.get(method).get('unit'),
                                                 method[1].title())
         )
    
results

Score is 4620.867554 kg SO2-Eq for category Acidification
Score is 1451712.044700 kg CO2-Eq for category Climate Change
Score is 305424.681945 kg 1,4-DCB-Eq for category Ecotoxicity: Freshwater
Score is 541236955.229911 kg 1,4-DCB-Eq for category Ecotoxicity: Marine
Score is 4622.939915 kg 1,4-DCB-Eq for category Ecotoxicity: Terrestrial
Score is 35589696.272327 megajoule for category Energy Resources: Non-Renewable
Score is 1131.355488 kg PO4-Eq for category Eutrophication
Score is 101618546.054719 kg 1,4-DCB-Eq for category Human Toxicity
Score is 2.326668 kg Sb-Eq for category Material Resources: Metals/Minerals
Score is 0.053242 kg CFC-11-Eq for category Ozone Depletion
Score is 426.951361 kg ethylene-Eq for category Photochemical Oxidant Formation


[('Acidification', 4620.867554305927, 'kg SO2-Eq'),
 ('Climate Change', 1451712.0447004328, 'kg CO2-Eq'),
 ('Ecotoxicity: Freshwater', 305424.68194525054, 'kg 1,4-DCB-Eq'),
 ('Ecotoxicity: Marine', 541236955.2299111, 'kg 1,4-DCB-Eq'),
 ('Ecotoxicity: Terrestrial', 4622.939915413022, 'kg 1,4-DCB-Eq'),
 ('Energy Resources: Non-Renewable', 35589696.272327244, 'megajoule'),
 ('Eutrophication', 1131.3554877978527, 'kg PO4-Eq'),
 ('Human Toxicity', 101618546.0547189, 'kg 1,4-DCB-Eq'),
 ('Material Resources: Metals/Minerals', 2.3266684801435487, 'kg Sb-Eq'),
 ('Ozone Depletion', 0.053242043984428895, 'kg CFC-11-Eq'),
 ('Photochemical Oxidant Formation', 426.9513612295997, 'kg ethylene-Eq')]

In [25]:
# We can also see the values stores in the list 'results'
results

[('Acidification', 4620.867554305927, 'kg SO2-Eq'),
 ('Climate Change', 1451712.0447004328, 'kg CO2-Eq'),
 ('Ecotoxicity: Freshwater', 305424.68194525054, 'kg 1,4-DCB-Eq'),
 ('Ecotoxicity: Marine', 541236955.2299111, 'kg 1,4-DCB-Eq'),
 ('Ecotoxicity: Terrestrial', 4622.939915413022, 'kg 1,4-DCB-Eq'),
 ('Energy Resources: Non-Renewable', 35589696.272327244, 'megajoule'),
 ('Eutrophication', 1131.3554877978527, 'kg PO4-Eq'),
 ('Human Toxicity', 101618546.0547189, 'kg 1,4-DCB-Eq'),
 ('Material Resources: Metals/Minerals', 2.3266684801435487, 'kg Sb-Eq'),
 ('Ozone Depletion', 0.053242043984428895, 'kg CFC-11-Eq'),
 ('Photochemical Oxidant Formation', 426.9513612295997, 'kg ethylene-Eq')]

In [26]:
#Creates a table with the results
results_df = pd.DataFrame(results, columns=["Impact Category", "Score", "Unit"])
results_df = results_df.set_index("Impact Category")
results_df

,Score,Unit
Impact Category,,
Acidification,4.620868e+03,kg SO2-Eq
Climate Change,1.451712e+06,kg CO2-Eq
Ecotoxicity: Freshwater,3.054247e+05,"kg 1,4-DCB-Eq"
Ecotoxicity: Marine,5.412370e+08,"kg 1,4-DCB-Eq"
Ecotoxicity: Terrestrial,4.622940e+03,"kg 1,4-DCB-Eq"
Energy Resources: Non-Renewable,3.558970e+07,megajoule
Eutrophication,1.131355e+03,kg PO4-Eq
Human Toxicity,1.016185e+08,"kg 1,4-DCB-Eq"
Material Resources: Metals/Minerals,2.326668e+00,kg Sb-Eq
